# DYSKUSJA

PYTANIE: Czego potrzebujemy?
ODPOWIEDZ: Uczenie nienadzorowanego!

PYTANIE: Czemu?
ODPOWIEDZ: ...

PYTANIE: Czego użyjemy?
ODPOWIEDZ: Używamy GAN.

PYTANIE: Czemu GAN?
ODPOWIEDZ: Ponieważ ogólna zasada jest taka, że próbujemy coś wygenerować i mamy detykowanego agenta, który mówi czy to się udało czy nie.
Brzmi ok do uczenia generowania obrazów już istniejących.

PYTANIE: W takim razie jak to działa że tworzy nowe obrazy?
ODPOWIEDZ: Mamy w sumie dwa modele, jeden ma generować np. kwiatki, a drugi ma mówić czy to co dostaje do kwiatek. Jak model rozpoznający kwiatki powie "to nie kwiatek" to model generujący się updatuje, a jak model rozpoznający powie "to totalnie jest kwiatek" na to co dał model generujący, to model rozpoznający się updatuje i tak w kółko...
Na sam koniec mamy model, który dostając obraz na wejście "przewiduje" co może być na wyjściu, czyli na nasze, będzie tworzył obraz jaki "być powinien" na bazie tego czego się nauczył.

PYTANIE: A jakie sieci w tym GAN pan ma?
ODPOWIEDZ: convolutional neural network (CNN)

PYTANIE: Czemu?
ODPOWIEDŹ: CNN mogą uczyć się na "historii" czyli trochę tak jakby to był film o kwiatkach i model będzie przewidywał następną klatkę tego filmu o kwiatkach. Żeby to zrobić musi coś wygenerować i nam zaproponować. To tutaj mamy część "twórczą".


# SCHEMAT ROZWIĄZANIA

Narazie ta część bazuje/jest z książki "Deep Learning Praca z językiem Python i biblioteką Kares" autorstwa Francois Chollet:
1. Sieć generatora "generator" mapuje wektor o kształcie "latent_dim" na obraz o kształcie (32, 32, 3).
2. Sieć dyskryminatora "discriminator" mapuje obraz o kształcie (32, 32, 3) na binarną wartość określającą prawdopodobieństwo tego, że obraz jest prawdziwy.
3. Sieć "gan" tworzy łańcuch  skłądajaćy się z generatora i dyskryminatora (gan(x)=discriminator(generator(x))). SIeć gan mapuje wektory niejawnej przestrzeni na oceny realizmu wystawiane przez dyskryminator.
4. Trenujemy dyskryminator przy użyciu przykładów prawdziwych i wygenerowanych przez generator, oznaczonych etykietami, tak jakbyśmy trenowali zwykły model klasyfikacji obrazów.
5. W celu wytrenowania generatora korzystamy z gradientów wag generatora w odniesieniu do straty modelu "gan". 
Inaczej (rozwijajac): W zwiazku z tym każdy krok trenowania ma modyfikować wagi generatora tak, aby zwiększyć prawdopodobieństwo zaklasyfikowania wygenerowanych obrazów jako prawdziwych. 
Inaczej (upraszczająć): Trenujemy generator tak by był w stanie oszukać dyskryminator.

# GENERATOR

In [7]:
import keras
from keras import layers
import numpy as np

# Zmienne opisujące obrazy
latent_dim = 32
heigh = 32
width = 32
channels = 3

# INPUT LAYER: Warstwa wejściowa generatora
generator_input = keras.Input(shape=(latent_dim,))

# HIDDEN LAYER: Pozostałe warstwy modelu generatora
# CO ROBIMY? 
# BO: Zmieniamy obiekt wejściowy w 128 kanałową mapę cech 16x16
# CZEMU TO TU JEST?
# BO: 
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

# CO ROBIMY?
# BO: Standardowo trenujemy "przewidywanie" jakie powinno być "wyjście" (jak w klatkach filmowych)
# CZEMU TO TU JEST?
# BO: 
# CZEMU CONV2D?
# CZMEU PADIDNG? CZEMU PADDING "SAME"?
# CZEMU TAKIE WARTOŚĆI?
x = layers.Conv2D(256, 5, padding='same')(x)
X = layers.LeakyReLU()(x)

# PO CO TO?
x = layers.Conv2D(256, 4, strides=2, padding='same')(x)  # Zwiększenie rozmiaru do 32x32
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# OUTPUT LAYER: Warstwa wyjściwowa generatora, która daje nam obraz. 
# Tworzy instancję generatora, która mapuje obiekt wejściowy o kształcie (latent_dim,) na obraz o kształcie (32, 32, 3)
# CZEMU TANH? JAK TO DZIAŁA?
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)  # Generuje jednokanałową mapę cech o rozmiarze 32x32 (rozmiar ten jesy taki sam jak rozmiar obraz,ow wchodzących w skład zbioru CIFAR10)
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_1 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32768)             0         
                                                                 
 reshape_1 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 256)       819456    
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 256)         1048832   
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 8, 8, 256)         0     

# NOTATKI
## Rule of thumbs
1. Ostatnia warstwa to *tanh*.
2. Próbkowanie punktów za pomocą rozkłądu Gaussa.
3. Dodajemy dużo losowości do procesu losowania (bo GAN ma tendencję do wpadania w optimum lokalne, BARDZO), np. drop wag i szum etykiet.
4. Rzadkie gradienty są fe. Co robi rzadki gradient? Np. Maxpooling i ReLU. Wiec zamiast maxpooling dajemy krokową konwolucję. Zamiast ReLU dajemy LeakyReLU.
5. Zawsze gdy używamy Conv2DTranpose lub Conv2D zastosujemy rozmiar jądra podzielony przez rozmiar kroku. POmoże nam to uniknąć artefaktów takich jak "szachownica" na generowanym obrazie.